In [ ]:
!pip install langchain openai

In [ ]:
!pip install langchain-experimental

In [ ]:
!pip install langchain openai langchain-experimental

In [ ]:
!pip install -U langchain langgraph google-api-python-client langchain[google-genai]

In [ ]:
!pip install langchain openai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
import os
import openai
from langchain.agents import Tool
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_types import AgentType
from langchain.chains.llm_math.base import LLMMathChain

In [ ]:
import os
import openai
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.agents import Tool
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import datetime

# Path to your service account key file
SERVICE_ACCOUNT_FILE = '/content/.json'

# Scopes required for Google Calendar
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

# Authenticate
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Build the service
service = build('calendar', 'v3', credentials=credentials)

# Get upcoming events
now = datetime.datetime.utcnow().isoformat() + 'Z'
events_result = service.events().list(
    calendarId='primary', timeMin=now,
    maxResults=5, singleEvents=True,
    orderBy='startTime').execute()
events = events_result.get('items', [])

for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])

In [ ]:
def get_upcoming_events():
    from google.oauth2 import service_account
    from googleapiclient.discovery import build
    import datetime

    SERVICE_ACCOUNT_FILE = 'credentials.json'
    SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('calendar', 'v3', credentials=credentials)

    now = datetime.datetime.utcnow().isoformat() + 'Z'
    events_result = service.events().list(
        calendarId='primary', timeMin=now,
        maxResults=5, singleEvents=True,
        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        return "No upcoming events found."

    response = ""
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        summary = event.get('summary', 'No title')
        response += f"{start}: {summary}\n"
    return response.strip()

In [ ]:
from langchain.agents import Tool

calendar_tool = Tool(
    name="Google Calendar",
    func=lambda _: get_upcoming_events(),
    description="Use this to check upcoming calendar events."
)

In [ ]:
!pip install -U langchain langchain-community google-auth google-api-python-client

import os
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI

# Optional: Set API key environment variables (if not set elsewhere)
os.environ["GOOGLE_API_KEY"] = "API key"  # Replace with actual key

# 1. Initialize the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# 2. Define a dummy calendar tool for testing (replace with real implementation)
def dummy_calendar_tool(query: str) -> str:
    return "You have a meeting today at 3 PM."

calendar_tool = Tool(
    name="GoogleCalendarTool",
    func=dummy_calendar_tool,
    description="Useful for checking calendar events."
)

# 3. Initialize the agent
agent = initialize_agent(
    tools=[calendar_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 4. Run agent with a user prompt
response = agent.run("Do I have any meetings today?")
print("\nFinal Response:\n", response)